In [2]:
#import needed libraries
import pandas as pd
import numpy as np
from pandas import DataFrame, Series
import json
import psycopg2
import os

In [3]:
def get_connection():
    conn = {}
    with open('../db_connection/db_config.json') as my_file:
        db_conf = json.load(my_file)
    conn_str = "host={} dbname={} user={} password={}".format(
                db_conf['host'], db_conf['database'], 
                db_conf['user'], db_conf['passw'])
    try:
        conn = psycopg2.connect(conn_str)
        return conn
    except:
        return 'error in connecting to database'

In [4]:
get_connection()

<connection object at 0x7f7eb62e8668; dsn: 'host=41.89.94.68 password=xxx user=boa_user dbname=mohdsl', closed: 0>

In [5]:
#get a specific county
def get_county_code_by_id(county_id):
    conn = get_connection()
    all_counties = pd.DataFrame()
    for chunk in pd.read_sql('SELECT * FROM common_county', con=conn, chunksize=10000):
        all_counties = all_counties.append(chunk)

    all_counties = all_counties[all_counties['id']==county_id]
    
    return all_counties

In [6]:
get_county_code_by_id('866cc286-8201-493e-a4db-a9b2a5f64f8e')

,id,county_sk,deleted,active,name,code,_datecreated,_eventexecutionkey,_lastaction,_mergekey,_currentflag,_effectivefrom,_effectiveto,_sourcesystemcode
46,866cc286-8201-493e-a4db-a9b2a5f64f8e,47,False,True,NAIROBI,47,2016-01-11,None,None,None,1,1900-01-01,2199-12-31,None


In [7]:
def get_county_constituency_codes(county_id):
    '''return codes for constituencies in county'''
    conn = get_connection()
    all_constituencies = DataFrame()

    for chunk in pd.read_sql('SELECT * FROM common_constituency', con=conn, chunksize=10000):
        all_constituencies = all_constituencies.append(chunk)
    
    county = get_county_code_by_id(county_id)
    county = county.rename(index=str, columns={"id": "county_id", 'name': 'county_name'})
    county_constituencies = pd.merge(all_constituencies,county, on='county_id')
    county_constituencies = county_constituencies[['name','id','county_id']]
    
    return county_constituencies

In [8]:
get_county_constituency_codes('866cc286-8201-493e-a4db-a9b2a5f64f8e')

,name,id,county_id
0,WESTLANDS,66eababf-7185-4479-919a-3448838d2f4f,866cc286-8201-493e-a4db-a9b2a5f64f8e
1,DAGORETTI NORTH,13185a56-ed1d-471a-8741-8b0e614a3bca,866cc286-8201-493e-a4db-a9b2a5f64f8e
2,DAGORETTI SOUTH,f11dc857-3bfc-46a5-ac32-23e11afd9f1c,866cc286-8201-493e-a4db-a9b2a5f64f8e
3,LANGATA,3eb79ef2-a75f-482f-a5b4-1cd12a312372,866cc286-8201-493e-a4db-a9b2a5f64f8e
4,KIBRA,504d12d7-038d-4b58-826e-4b5adb03c738,866cc286-8201-493e-a4db-a9b2a5f64f8e
5,ROYSAMBU,2150758f-a16d-46e2-9847-64ad225ddd69,866cc286-8201-493e-a4db-a9b2a5f64f8e
6,KASARANI,e621888a-ded7-4bc2-810d-ebb39466e01c,866cc286-8201-493e-a4db-a9b2a5f64f8e
7,RUARAKA,c357fee9-cb98-4451-b99e-e1ee5c59811f,866cc286-8201-493e-a4db-a9b2a5f64f8e
8,EMBAKASI SOUTH,62b63d09-6d2b-4ee7-86c7-3390b41fd746,866cc286-8201-493e-a4db-a9b2a5f64f8e
9,EMBAKASI NORTH,1c34f5c6-cc67-45d3-9ba7-21d064d36f54,866cc286-8201-493e-a4db-a9b2a5f64f8e


In [9]:
def get_constituency_wards(constituency_id):
    '''return ids for wards in a constituency'''
    conn = get_connection()
    all_wards = DataFrame()

    for chunk in pd.read_sql('SELECT * FROM common_ward', con=conn, chunksize=10000):
        all_wards = all_wards.append(chunk)
    
    

    constituency = constituencies.get_constituency_by_id(constituency_id)
    constituency = constituency.rename(index=str, columns={'id':'constituency_id', 'name': "constituency_name"})
    # import pdb
    # pdb.set_trace()

    constituency_wards = pd.merge(all_wards,constituency, on='constituency_id')
    constituency_wards = constituency_wards[['name','id','constituency_id']]

    return constituency_wards.to_json(orient='records')

In [10]:
#get all the facilities in a county
def get_county_facilities(county_id):
    conn = get_connection()
    county = get_county_code_by_id(county_id)
    
    #get constituencies in the county
    constituencies = get_county_constituency_codes(county_id)
    constituencies = constituencies.rename(index=str, columns={'id':'constituency_id', 'name': "constituency_name"})
    
    #get all the wards for the county
    all_wards = DataFrame()
    for chunk in pd.read_sql('SELECT * FROM common_ward', con=conn, chunksize=10000):
        all_wards = all_wards.append(chunk)
    
    #merge with the consitituencies
    county_wards = pd.merge(all_wards,constituencies,on='constituency_id')
    county_wards = county_wards.rename(index=str, columns={'id': 'ward_id', 'name': 'ward_name'})
    
    #get all facilieits and merge
    all_facilities = DataFrame()
    for chunk in pd.read_sql('SELECT * FROM facilities_facility', con=conn, chunksize=10000):
        all_facilities = all_facilities.append(chunk)
    
    
    county_facilities = pd.merge(all_facilities,county_wards, on='ward_id')
    return county_facilities

In [11]:
county_facilities = get_county_facilities("866cc286-8201-493e-a4db-a9b2a5f64f8e")

In [12]:
#get county summaries
def get_county_summary(county_id):
    '''return a summary of beds and cots per county'''
    county_facilities = get_county_facilities(county_id)
    
    county_facilities['number_of_facilities'] = 1
    county_summary = county_facilities.groupby(['constituency_id','constituency_name'],as_index=False).sum()[
        ['number_of_beds','number_of_cots','number_of_facilities','constituency_id','constituency_name']]
    
    return county_summary

In [13]:
get_county_summary("866cc286-8201-493e-a4db-a9b2a5f64f8e")

,number_of_beds,number_of_cots,number_of_facilities,constituency_id,constituency_name
0,2132,590,45,08b1fda1-c9b8-4ddf-ae05-c1c3abccb142,MATHARE
1,124,13,34,13185a56-ed1d-471a-8741-8b0e614a3bca,DAGORETTI NORTH
2,64,6,30,1c34f5c6-cc67-45d3-9ba7-21d064d36f54,EMBAKASI NORTH
3,30,6,54,2150758f-a16d-46e2-9847-64ad225ddd69,ROYSAMBU
4,986,185,53,307df120-3438-4e29-9f47-848124ee452d,KAMUKUNJI
5,765,2,79,3eb79ef2-a75f-482f-a5b4-1cd12a312372,LANGATA
6,328,55,137,497d61ec-dcf5-4ce6-a03e-c46c99e16992,STAREHE
7,2158,541,86,504d12d7-038d-4b58-826e-4b5adb03c738,KIBRA
8,205,25,43,58a69438-cec6-4f33-b531-4898a6595c80,MAKADARA
9,149,22,22,5a6e4b7b-96a2-44c8-a145-857e0f583aad,EMBAKASI WEST


In [14]:
county_facilities.groupby(['constituency_id','constituency_name'],as_index=False).sum()[['number_of_beds','number_of_cots','facility_count','constituency_id','constituency_name']]

KeyError: "['facility_count'] not in index"

In [ ]:
pd.read_sql("""
                SELECT common_constituency.name, common_constituency.id
                FROM common_constituency, common_county
                WHERE common_constituency.county_id = common_county.id
                AND common_county.id = '%s' """ %('866cc286-8201-493e-a4db-a9b2a5f64f8e'), get_connection())

In [ ]:
pd.read_sql("""
                SELECT common_ward.name, common_ward.id
                FROM common_constituency, common_ward
                WHERE common_ward.constituency_id = common_constituency.id
                AND common_constituency.id = '%s' """ %('2150758f-a16d-46e2-9847-64ad225ddd69'), get_connection())

In [16]:
pd.read_sql('''
                SELECT common_ward.name, common_ward.id
                FROM common_ward
                WHERE common_ward.constituency_id in ( '66eababf-7185-4479-919a-3448838d2f4f', 'f11dc857-3bfc-46a5-ac32-23e11afd9f1c' ) '''  , get_connection())

,name,id


In [ ]:
pd.read_sql('''
                SELECT common_ward.name, common_ward.id
                FROM common_constituency, common_ward
                WHERE common_ward.constituency_id = common_constituency.id
                AND common_constituency.id = 
                    ( SELECT common_constituency.id
                        FROM common_constituency, common_county
                        WHERE common_constituency.county_id = common_county.id
                        AND common_county.id = '%s' 
                        LIMIT 1) ''' %('866cc286-8201-493e-a4db-a9b2a5f64f8e') , get_connection())

In [ ]:
pd.read_sql( '''  
                SELECT a.name, b.id, c.name
                FROM common_ward  AS a
                JOIN common_constituency AS b ON a.constituency_id = b.id
                JOIN common_county AS c ON b.county_id = c.id
                WHERE c.id = '%s'
                '''%('866cc286-8201-493e-a4db-a9b2a5f64f8e') , get_connection())

In [ ]:
pd.read_sql('''( SELECT common_constituency.id
                        FROM common_constituency, common_county
                        WHERE common_constituency.county_id = common_county.id
                        AND common_county.id = '866cc286-8201-493e-a4db-a9b2a5f64f8e' )''' , get_connection())